In [1]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np

# Para visualizar el progreso:
import tqdm

# FUNCIONES SCRAPING:

- URLS

In [11]:
def iniciar_scraper(user_agent, scrape_urls):
    
    # Options:
    opts = Options()
    opts.add_argument("--disable-extensions")
    opts.add_argument(f"User-Agent={user_agent}")
    
    urls_paginas_perfumes = []
    
    inicio = 1
    for url in scrape_urls:
        # Initialize:
        browser = webdriver.Chrome(options=opts)
        browser.maximize_window()
        sleep(2)

        browser.get(url)
        sleep(5)###########################################################################
        
        #Acepto cookies:
        if inicio == 1:            
            browser.find_element(By.ID, "onetrust-accept-btn-handler").click()
            sleep(6)########################################################################

        soup = BeautifulSoup(browser.page_source, "html.parser")
        
        num_paginas = int(soup.find("div", class_= "styled__FooterWrapper-sc-1ecav21-0 komnRu").find_all("span")[-1].text)        
        if inicio == 1:
            urls_paginas = [f"{url}?f={num_pagina}-1-55544-55549" for num_pagina in range(1,num_paginas)]
        if inicio == 2:
            urls_paginas = [f"{url}?f={num_pagina}-1-55544-55545" for num_pagina in range(1,num_paginas)]
            
        urls_paginas_perfumes.extend(urls_paginas)
        
        inicio += 1
        browser.close()  
        
    ############################################################################################################    
      
    return pd.DataFrame(urls_paginas_perfumes, columns= ["url"])

- FUNCIÓN INFO

In [11]:
def scrape_info(user_agent, url):
   
    
    # Options:
    opts = Options()
    opts.add_argument("--disable-extensions")
    opts.add_argument(f"User-Agent={user_agent}")
    
    
    browser = webdriver.Chrome(options=opts)
    #browser.set_window_position(-2000,0)
    sleep(2)
    
    browser.get(url)
    sleep(5)
    
    # Acepto cookies:
    browser.find_element(By.ID, "onetrust-accept-btn-handler").click()
    sleep(2)
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    attrs= {"data-testid": "product-container"}
    productos = soup.find_all("div", attrs= attrs)
    
    cat = soup.find("h1", class_= "styled__StyledPageTitle-sc-1ddrixz-1 dtvZwa").text
    
    info_productos = []
    
    for producto in productos:
        
        categoria = cat
        
        try:
            marca = producto.find("div", class_= "sc-fhzFiK cnoxFk").find("h2").text
        except:
            marca = "Sin marca"
            
        try:
            nombre = producto.find("div", class_= "sc-fhzFiK cnoxFk").find("h3").text
        except:
            nombre = "Sin nombre"
            
        try:
            tipo = producto.find("div", class_= "sc-fhzFiK cnoxFk").find("p").text
        except:
            tipo = "Sin categoria"       
                 
        try:
            attrs_1= {"data-testid": "product-price"}
            attrs_2= {"data-testid": "price-component"}
            precio_con_desc = producto.find("div", attrs= attrs_1).find("span", attrs= attrs_2).text
        except:
            precio_con_desc = "Sin precio"
            
        try:
            descuento = producto.find("span", class_= "styled__DiscountValue-sc-1b3ggfp-1 llWxOW").text
        except:
            descuento = "Sin descuento"
        
        try:
            url_producto = ("https://www.notino.es" + producto.find("a")["href"])
            #################################################
            # Initialize:
            browser_prod = webdriver.Chrome(options=opts)
            sleep(2)
            browser_prod.get(url_producto)
            sleep(4)            
            browser_prod.find_element(By.ID, "onetrust-accept-btn-handler").click()
            sleep(2)
            
            ############################### CAMBIA ESTO
            soup_prod = BeautifulSoup(browser_prod.page_source, "html.parser")
            
            attrs_3= {"id": "pdHeader"}
            attrs_4= {"id": "pdSelectedVariant"}
            attrs_5= {"data-testid": "originalPriceLineThroughWrapper"}
            
            valoracion = soup_prod.find("div", attrs=attrs_3).find_all("a")[1]["title"]
            num_valoraciones = soup_prod.find("div", attrs=attrs_3).find("span", class_= "sc-kgTSHT bSWHSL").text.replace("×", "")
            precio_sin_desc = soup_prod.find("div", attrs= attrs_4).find("span", attrs= attrs_5).find("span").text
            sleep(2)
            browser_prod.close()
            #################################################
            
        except:
            valoracion = "Sin valoracion"
            num_valoraciones = "Sin valoraciones"
            precio_sin_desc = "Es el precio con desc"
        
        info_productos.append([categoria, marca, nombre, tipo, precio_sin_desc, descuento, precio_con_desc, valoracion, num_valoraciones])
        
    browser.close()
    return info_productos

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

# Scrapeo URLS

In [5]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

In [47]:
perfumes_hombre = "https://www.notino.es/perfumes-hombre/"
perfumes_mujer = "https://www.notino.es/perfumes-mujer/"

busqueda = [perfumes_hombre, perfumes_mujer]

In [48]:
df_urls_perfumes = iniciar_scraper(path, user_agent, busqueda)

C:\Users\regue\AppData\Local\Temp\ipykernel_24552\454601998.py:14: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path = path_driver, chrome_options=opts)


## CSV

In [52]:
# df_urls_perfumes.to_csv("urls_perfumes_notino.csv", index="False", sep= ",")

# Scrapeo la info de todos los productos de cada página:

In [6]:
df_urls_perfumes = pd.read_csv("urls_perfumes_notino.csv")

In [12]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

info_productos = []
for url in tqdm.tqdm(df_urls_perfumes["url"].values[200:225]):

    info = scrape_info(user_agent, url)
    info_productos.extend(info)

100%|██████████| 25/25 [2:31:10<00:00, 362.82s/it]  


In [16]:
df_info_productos = pd.DataFrame(info_productos, columns= ["categoria", "marca", "nombre", "tipo", "precio_sin_desc", "descuento", "precio_con_desc", "valoracion", "num_valoraciones"] )

## CSV

In [18]:
df_info_productos.to_csv("datos_perfumes_notino_200_225.csv", index="False", sep= ",")